In [ ]:
import pandas as pd
all_reviews = pd.read_csv("all_reviews.csv")
# all_reviews must include:
# product_name, review_text, rating, retailer, etc.

# Ensure review_text is string
all_reviews["review_text"] = all_reviews["review_text"].astype(str)

# Group all reviews per product and concatenate into one giant document
nlp_merged = (
    all_reviews
        .groupby("product_name")
        .agg({
            "review_text": lambda x: " ".join(x),   # merge all reviews into one big text
            "rating": "mean",                       # average rating
            "product_name": "count"                 # count of reviews
        })
        .rename(columns={
            "review_text": "merged_reviews",
            "rating": "avg_rating",
            "product_name": "num_reviews"
        })
        .reset_index()
)

nlp_merged


,product_name,merged_reviews,avg_rating,num_reviews
0,Armani Luminous Foundation,I love this foundation 😍ıt is look so fresh.ıt...,4.350000,200
1,BYOMA Moisturizing Rich Cream,Bought this moisturizer to add to my morning s...,4.700000,110
2,Clinique Dramatically Different Moisturizing L...,Keeps coming back to this OG lotion. The only ...,4.209524,210
3,Dior Forever Skin Glow,Nice hydrating foundation. A thin layer goes a...,4.083744,203
4,Drunk Elephant Lala Retro Whipped Moisturizer,"I love how portable this mini is, it’s great f...",4.233333,210
5,Estée Lauder Double Wear,I don’t like the fact that I have purchased at...,4.390000,300
6,Estée Lauder Revitalizing Supreme+ Global Anti...,This doesn’t absorb and makes me look really g...,4.195238,210
7,Fenty Pro Filt'r Soft Matte Foundation,"best foundation in the market, love it!! best ...",3.785000,200
8,Kiehl’s Ultra Facial Moisturizing Cream with S...,This doesn’t absorb and makes me look really g...,4.271429,210
9,Lancôme Rénergie H.P.N. 300-Peptide Cream,This doesn’t absorb and makes me look really g...,4.314286,210


In [ ]:
nlp_merged.to_csv("nlp_merged_reviews.csv", index=False)
print("Saved → nlp_merged_reviews.csv")


Saved → nlp_merged_reviews.csv


In [ ]:
pip install transformers sentencepiece torch nltk


In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import pandas as pd

df = pd.read_csv("nlp_merged_reviews.csv")
df.head()


,product_name,merged_reviews,avg_rating,num_reviews
0,Armani Luminous Foundation,I love this foundation 😍ıt is look so fresh.ıt...,4.350000,200
1,BYOMA Moisturizing Rich Cream,Bought this moisturizer to add to my morning s...,4.700000,110
2,Clinique Dramatically Different Moisturizing L...,Keeps coming back to this OG lotion. The only ...,4.209524,210
3,Dior Forever Skin Glow,Nice hydrating foundation. A thin layer goes a...,4.083744,203
4,Drunk Elephant Lala Retro Whipped Moisturizer,"I love how portable this mini is, it’s great f...",4.233333,210


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from transformers import pipeline

nltk.download("punkt")

# Load merged review dataset
df = pd.read_csv("nlp_merged_reviews.csv")

# Load zero-shot classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

labels = ["benefit", "disadvantage", "neutral"]

product_data = {}

for idx, row in df.iterrows():
    product = row["product_name"]
    text = row["merged_reviews"]
    sentences = sent_tokenize(text)

    benefits = []
    disadvantages = []

    for s in sentences:
        result = classifier(s, labels)
        top_label = result["labels"][0]

        if top_label == "benefit":
            benefits.append(s)
        elif top_label == "disadvantage":
            disadvantages.append(s)

    product_data[product] = {
        "benefits": benefits,
        "disadvantages": disadvantages
    }

# Save dictionary for later use
import pickle
with open("product_benefits_disadvantages.pkl", "wb") as f:
    pickle.dump(product_data, f)

print("EXTRACTION COMPLETE ✓")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


EXTRACTION COMPLETE ✓


In [ ]:
product = "Lancôme Rénergie H.P.N. 300-Peptide Cream"

benefits = product_data[product]["benefits"][:10]
disadvantages = product_data[product]["disadvantages"][:10]

print("=== BENEFITS ===")
for b in benefits:
    print("-", b)

print("\n=== DISADVANTAGES ===")
for d in disadvantages:
    print("-", d)


=== BENEFITS ===
- Not sure why the positive reviews.
- The most incredible moisturizer I’ve ever used!
- Very moisturizing and after using for only a short time, I’ve noticed a huge difference in how my face and neck feel and look!
- It’s worth every dime.
- I’ve been using the Estée Lauder Revitalizing Supreme+ Youth Power Creme SPF 25 for a while now, and it truly feels like a little luxury in my skincare routine.
- The texture is rich and silky, yet it absorbs quickly without leaving my skin greasy.
- Right after applying, my face feels smooth, hydrated, and has a healthy glow that lasts throughout the day.
- I also love that it has SPF 25, so I can use it as my daytime moisturizer and know I’m getting extra protection.
- Over time I noticed my skin looking a bit firmer and more even, especially around the jawline and smile lines.
- It also works really well under makeup, almost like a primer, keeping my foundation fresh without making it cakey.

=== DISADVANTAGES ===
- This doesn’